# Creating, saving, and loading an individual task

### Imports

In [7]:
from cognitive.task_generator import TemporalTask
from cognitive import stim_generator as sg
from cognitive import task_generator as tg
from cognitive import constants as const

### Task Creation

- Here we create a task where the goal is to correctly compare the locations of two stimuli (objects)
    - We use the TemporalTask class as a parent class to build from. (All custom tasks must inherit from this TemporalTask)

In [8]:
class CompareLocTemporal(TemporalTask):
    """Compare location between two objects."""

    def __init__(self, whens=None, first_shareable=None):
        """
            :param whens: the list of two frame names to compare stimuli location between
        """

        # Initialize Class with parent class
        super(CompareLocTemporal, self).__init__(whens=whens)

        # Select the specified frames
        objs1 = tg.Select(when=self.whens[0])
        objs2 = tg.Select(when=self.whens[1])

        # Get the locations of stimuli within each frame
        a1 = tg.GetLoc(objs1)
        a2 = tg.GetLoc(objs2)

        # Set operator to check if they're the same location
        self._operator = tg.IsSame(a1, a2)

        # 
        self.n_frames = const.compare_when([when1, when2]) + 1

    @property
    def instance_size(self):
        return sg.n_sample_shape(2) * (sg.n_random_when()) ** 2


In [ ]:
max

In [9]:
[f'last{n}' for n in range(nback)]

NameError: name 'nback' is not defined

In [ ]:
task

# load_operator_json

In [ ]:
# once function to reconstruct graph from Task object is written show to how to save the graph as an image. 